In [2]:
import shutil, os, csv, itertools, glob
import math
import numpy as np
import pandas as pd
import pickle as pk
import time
from os import listdir
from os.path import isfile, join
import collections
import matplotlib.pyplot as plt
import random
import argparse
import gzip
from geopy.distance import distance
from datetime import datetime, timedelta

# Raw data downloading:
## 1. visit PEMS website http://pems.dot.ca.gov
## 2. Select District 7 LA/Ventura, click "Data Clearinghouse'
## 3. Select Type: Station Hour, District 7, download hourly traffic volume data, e.g., d07_text_station_hour_2018_01.txt.gz
## 4. Select Tpye: Station Metadata, District 7, download station meta data, e.g., d07_text_meta_2017_12_23.txt


# Process raw traffic volume data, only 2 hourly traffic volumes are missing

In [ ]:
mypath = "../../data/Pems_LA_volume/volume"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

res = []
for filename in onlyfiles:
    with gzip.open("../../data/Pems_LA_volume/volume/"+filename, 'rb') as f:
        for line in f:
            line = str(line)
            line = line[2:].split(',') 
            res.append(line[0:10])
            
res = pd.DataFrame(res)
res.columns = ['time', 'stationid', 'district', 'route', 'travel_dict', 'lanetype', 'station_length', 'samples', 'observed', 'total_flow']


In [ ]:
# lane type
#CD (Coll/Dist)
#CH (Conventional Highway)
#FF (Fwy-Fwy connector)
#FR (Off Ramp)
#HV (HOV)
#ML (Mainline)
#OR (On Ramp)

# Sum of 5-minute flows over the hour. 
#Note that the basic 5-minute rollup normalizes flow by the number of good samples received from the controller.
#Veh/Hour

In [ ]:
# delete empty flow
res = res[res['total_flow'] != '']

res_ML = res[res['lanetype'] == 'ML']

res_ML_group = res_ML.groupby(['stationid']).size().reset_index()
res_ML_group.columns = ['stationid', 'count']

res_ML_13102 = res_ML_group[res_ML_group['count'] == 13102] # most of sensors miss 2 hourly traffic volume, should be 13104 / 24
print (res_ML_13102.shape) # 1858 sensors have 13102 records

In [ ]:
final_res = []

def hourly_it(start, finish):
     while finish > start:
             yield start
             start = start + timedelta(hours=1)
             
start = datetime(2018, 1, 1, 0)
finish = datetime(2019, 7, 1, 0)
for hour in hourly_it(start, finish):
     final_res.append(hour)

In [ ]:
final_res = pd.DataFrame(final_res)
final_res.columns = ['new_time']

In [ ]:
column_names = []
for stid in res_ML_13102['stationid']:
    res_ML_tem = res_ML[res_ML['stationid'] == stid]
    res_ML_tem.loc[:, 'new_time'] = pd.to_datetime(res_ML_tem['time'])
    final_res = pd.merge(final_res, res_ML_tem[['new_time', 'total_flow']], on='new_time', how='left')
    column_names.append(stid)
    print (stid)


In [ ]:
final_res.columns = ['new_time']+ column_names

In [ ]:
# whether it is NaN or not
final_res[final_res.iloc[:, 1] != final_res.iloc[:, 1]]

In [ ]:
# there are two time points the data of which are all missing for all sensors
final_res.iloc[1658, 1:] = (final_res.iloc[1657, 1:].astype(int) + final_res.iloc[1659, 1:].astype(int)) / 2
final_res.iloc[10394, 1:] = (final_res.iloc[10393, 1:].astype(int) + final_res.iloc[10395, 1:].astype(int)) / 2

In [ ]:
final_res.to_csv('traffic_volume_district7.csv')

# Extract sensor information

In [5]:
sensor = []
f=open("../../data/Pems_LA_volume/sensor/d07_text_meta_2017_12_23.txt", "r")
lines = f.readlines()

for x in lines:
    sensor.append(x.split('\t'))

In [6]:
sensor = pd.DataFrame(sensor)
sensor.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,715898,5,S,7,37,40032,.71,117.28,33.880183,-118.021787,.43,ML,3,PHOEBE,2029,,,\n
1,715900,5,S,7,37,40032,1.06,117.63,33.882892,-118.026822,,OR,1,VALLEY VIEW,3255,,,\n
2,715901,5,N,7,37,40032,1.11,117.743,33.8834,-118.027451,,OR,1,VALLEY VIEW,3268,,,\n
3,715903,5,N,7,37,69154,1.56,118.193,33.886992,-118.034125,,OR,1,ALONDRA,3269,,,\n
4,715906,5,S,7,37,52526,3.28,119.85,33.900669,-118.059422,,OR,1,ROSECRANS,3252,,,\n


In [7]:
sensor.columns = ['ID', 'Fwy', 'Dir', 'District', 'County', 'City', 'State_PM', 'Abs_PM', 'Latitude', 'Longitude', 'Length', 'Type', 'Lanes', 'Name', 'User_ID_1', 'User_ID_2', 'User_ID_3', 'User_ID_4']

In [6]:
sensor_select = sensor[['ID', 'Latitude', 'Longitude']]

In [8]:
traffic_volume = pd.read_csv('../../data/Pems_LA_volume/traffic_volume_district7.csv')
traffic_volume.head()

,Unnamed: 0,new_time,715898,715918,715920,715929,715930,715933,715938,715944,...,775961,775975,775990,776019,776020,776052,776060,776066,776073,776081
0,0,2018-01-01 00:00:00,887.0,1045.0,2931.0,3303.0,3067.0,2763.0,2994.0,2622.0,...,91.0,216.0,3808.0,2405.0,3231.0,2461.0,3730.0,3752.0,2637.0,4122.0
1,1,2018-01-01 01:00:00,561.0,687.0,4558.0,5058.0,4559.0,3944.0,4417.0,3952.0,...,156.0,260.0,4294.0,3139.0,4047.0,3310.0,4808.0,5574.0,3641.0,6020.0
2,2,2018-01-01 02:00:00,483.0,543.0,3167.0,3527.0,3470.0,3158.0,3452.0,3119.0,...,68.0,177.0,3348.0,2311.0,3057.0,1998.0,3387.0,3340.0,2079.0,3593.0
3,3,2018-01-01 03:00:00,531.0,589.0,2126.0,2255.0,1808.0,1642.0,1834.0,1697.0,...,52.0,157.0,2457.0,1726.0,2400.0,1530.0,2884.0,2773.0,1595.0,3024.0
4,4,2018-01-01 04:00:00,1088.0,1263.0,1452.0,1284.0,1336.0,1270.0,1428.0,1307.0,...,48.0,148.0,1440.0,1417.0,1982.0,959.0,2048.0,1380.0,823.0,1523.0


In [9]:
sensor_res = []
volume_columns = []
i = 2
while i < len(traffic_volume.columns):
    has_loc_or_not = sensor_select[sensor_select['ID'] == traffic_volume.columns[i]].values[0]
    if has_loc_or_not[1] != '' and has_loc_or_not[2] != '': # lat and long exist
        sensor_res.append(has_loc_or_not)
        volume_columns.append(traffic_volume.columns[i])
    i = i + 1
    
# 1856 sensors left

In [17]:
sensor_res = pd.DataFrame(sensor_res)
sensor_res.columns = ['ID', 'Latitude', 'Longitude']
sensor_res.to_csv('sensor_location.csv', index=False)

In [18]:
volume_res = traffic_volume[volume_columns]
volume_res.to_csv('sensor_volume.csv', index = False)

# only pick the first 150 sensors as a small network, because of the computational limitation

In [ ]:
sensor_volume = pd.read_csv("../../data/Pems_LA_volume/sensor_volume.csv")
sensor_volume.head()

In [ ]:
sensor_volume.iloc[:, 0:150].to_csv("../../data/Pems_LA_volume/sensor_volume_150.csv")

In [8]:
sensor_location = pd.read_csv("../../data/Pems_LA_volume/sensor_location.csv")
sensor_location.head()

,ID,Latitude,Longitude
0,715898,33.880183,-118.021787
1,715918,33.933110,-118.091005
2,715920,33.938544,-118.094941
3,715929,33.971707,-118.123095
4,715930,33.971763,-118.122905


In [ ]:
sensor_location.iloc[0:150, :].to_csv("../../data/Pems_LA_volume/sensor_location_150.csv")

# calculate spatial distances as the adjacency matrix for DCRNN

In [ ]:
res = [] # for benchmakr model DCRNN

for i in range(150):
    row = []
    for j in range(150):
        dist = distance(sensor_loc.iloc[i, 1:], sensor_loc.iloc[j, 1:]).m
        row.append(dist)
        res.append([sensor_loc.iloc[i, 0], sensor_loc.iloc[j, 0], dist])
    
#print (res)

In [ ]:
res = pd.DataFrame(res)
res.columns = ['from', 'to', 'cost']
res.to_csv('distance_DCRNN.csv')